In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# this is the EM specific package for querying the EM data
from caveclient import CAVEclient


from meshparty import trimesh_io, trimesh_vtk
from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# mn_inputs = client.materialize.synapse_query(post_ids = 648518346488512137,timestamp=connectome_create.get_timestamp())

In [ ]:
# mn_inputs

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load Motor neuron table

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='edited')
pre_to_mn_df.shape

In [ ]:
cmap = utils.white_dense()

mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

conn_mat = np.log10(pre_to_mn_df.to_numpy()+1)

# TODO: split up the matrix on segments and types

fig = plt.figure(1, figsize = [15,25])
ax = sns.heatmap(conn_mat,xticklabels=lbls,cmap=cmap)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neuron SegIDS', fontsize =18)
plt.ylabel('Intersegmental pMNs', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

# This is basically randomly ordered

# Sort according to total number of inputs

In [ ]:
s = pre_to_mn_df.sum(axis=0)
pre_to_mn_df = pre_to_mn_df[s.sort_values(ascending=True).index]
pre_to_mn_df.shape

# Sort according to segment and function

In [ ]:
new_idx = utils.sort_segment_fcn_index(pre_to_mn_df.columns)
pre_to_mn_df = pre_to_mn_df[new_idx]

mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

conn_mat = np.log10(pre_to_mn_df.to_numpy()+1)

# TODO: split up the matrix on segments and types

fig = plt.figure(1, figsize = [15,25])
ax = sns.heatmap(conn_mat,xticklabels=lbls,cmap=cmap)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neuron SegIDS', fontsize =18)
plt.ylabel('Intersegmental pMNs', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

In [ ]:
mn_mi

In [ ]:
# Double check the order of the neurons
x = utils.xticks_from_mnmi(mn_mi)
x = utils.xticks_from_pools(mn_mi)
lbls = utils.mn_labels(mn_mi,depth='rank')
mat = pre_to_mn_df.to_numpy()
y = mat.sum(axis=0)

fig = plt.figure(1, figsize=(20, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
plt.sca(ax1)
plt.title('Number of descending neurons connected to each MN')
plt.xlabel('MN')
plt.ylabel('Number of inputs')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)

plt.savefig('./figpanels/total_inputs.svg',format='svg')

## Create ordered type specific neurons

In [ ]:
All = slice(None)

In [ ]:
### Motor neurons
# segID	motor	has_soma	sensory	 neck	local	
m_r_tup = (All,True,All,All,All,All)
mns = pre_to_mn_df.loc[m_r_tup,:]
mns = mns.loc[mns.sum(axis=1).sort_values(ascending=False).index,:]
print('mns shape: {}'.format(mns.shape))

### sensory neurons
# segID	motor	has_soma	sensory	 neck	local	
s_r_tup = (All,False,False,True,False,False)
sensory = pre_to_mn_df.loc[s_r_tup,:]
sensory = sensory.loc[sensory.sum(axis=1).sort_values(ascending=False).index,:]
print('sensory shape: {}'.format(sensory.shape))

### descending neurons
# segID	motor	has_soma	sensory	 neck	local	
descending = pre_to_mn_df.loc[(All,False,False,False,True,All),:]
descending = descending.loc[descending.sum(axis=1).sort_values(ascending=False).index,:]
print('descending shape: {}'.format(descending.shape))

### How many ascending neurons # 138
# segID	motor	has_soma	sensory	 neck	local	
ascending = pre_to_mn_df.loc[(All,False,True,False,True,False),:]
ascending = ascending.loc[ascending.sum(axis=1).sort_values(ascending=False).index,:]
print('ascending shape: {}'.format(ascending.shape))

### VNC neurons
# segID	motor	has_soma	sensory	 neck	local	
vncns = pre_to_mn_df.loc[(All,False,True,False,False,False),:]
vncns = vncns.loc[vncns.sum(axis=1).sort_values(ascending=False).index,:]
print('vncns shape: {}'.format(vncns.shape))

## strange motor neurons
sensmns = pre_to_mn_df.loc[(All,False,True,True,False,False),:]
sensmns = sensmns.loc[sensmns.sum(axis=1).sort_values(ascending=False).index,:]
print('sensmns shape: {}'.format(sensmns.shape))

(mns.shape[0]+sensory.shape[0]+descending.shape[0] + ascending.shape[0] + vncns.shape[0] + sensmns.shape[0] )

### Fragments

In [ ]:
# Neurons with a soma and not ascending
fragments = pre_to_mn_df.loc[(All,False,False,False,False,False),:].copy()
### VNC neurons
# segID	motor	has_soma	sensory	 neck	local	
fragments = pre_to_mn_df.loc[(All,False,False,False,False,False),:]
fragments = fragments.loc[fragments.sum(axis=1).sort_values(ascending=False).index,:]

total = fragments.sum(axis=1)
tot = total.cumsum()#/total.sum() # The fraction of total input from a given fragment
n = np.array([i for i in range(len(tot))])  # assign a number to neurons
frc = n/n.sum()

fig = plt.figure(1, figsize=(8, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

# ax1.scatter(x, y)
ax1.plot(n,tot)#, marker='o')
plt.sca(ax1)
plt.title('Total synapses from N fragments')
plt.ylabel('M synapses')
plt.xlabel('N fragments')
plt.show()
# locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)

In [ ]:
# # idx = fragments.index.to_frame()
utils.total_sort_save_df_csv(fragments,'fragment_list')

## Reassemble the connectivity matrix 

In [ ]:
ordered_pMN_to_MN_df = pd.concat([
    mns,
    descending,
    sensory,
    vncns,
    ascending,
    sensmns,
    fragments
])
print(ordered_pMN_to_MN_df.shape)

# Save the ordered_pMN_to_MN_df

In [ ]:
connectome_create.save_pre_to_mn_df(ordered_pMN_to_MN_df,ext='ordered')

In [ ]:
conn_mat = np.log10(ordered_pMN_to_MN_df.to_numpy()+1)

# TODO: split up the matrix on segments and types

fig = plt.figure(1, figsize = [15,25])
ax = sns.heatmap(conn_mat,xticklabels=lbls,cmap=cmap)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neuron SegIDS', fontsize =18)
plt.ylabel('Intersegmental pMNs', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()